In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import (
    Dense,
    Conv1D,
    Conv2D,
    MaxPooling1D,
    MaxPooling2D,
    Activation,
    LSTM,
    Embedding
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import set_random_seed


<IPython.core.display.Javascript object>

# Testing Convolution Layer

In [3]:
m = np.array([[[1, 1, 1], [10, 10, 10], [20, 20, 20], [20, 20, 20]]], dtype="float")


def conv_layer(m, filters: int, kernel_size: int, strides: int):
    set_random_seed(42)

    c = Conv1D(
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        name="conv_1d",
        use_bias=False,
    )

    print(m.shape)
    print(c.__call__(m))
    return c

<IPython.core.display.Javascript object>

## filters: 1
## kernel_size: 1
## strides: 1

In [4]:
c = conv_layer(m, filters=1, kernel_size=1, strides=1)

(1, 4, 3)
tf.Tensor(
[[[-0.10177284]
  [-1.0177282 ]
  [-2.0354564 ]
  [-2.0354564 ]]], shape=(1, 4, 1), dtype=float32)


<IPython.core.display.Javascript object>

In [5]:
# column i corresponds to filter i

weights = c.weights[0].numpy()
weights

array([[[ 0.40309322],
        [-0.14450324],
        [-0.36036283]]], dtype=float32)

<IPython.core.display.Javascript object>

In [6]:
for i in range(0, m.shape[1]):
    a = np.dot(m[0][i], weights[0])
    print(a)

[-0.10177284]
[-1.01772845]
[-2.0354569]
[-2.0354569]


<IPython.core.display.Javascript object>

## filters: 2
## kernel_size: 1
## strides: 1

In [7]:
c = conv_layer(m, filters=2, kernel_size=1, strides=1)

(1, 4, 3)
tf.Tensor(
[[[ -0.9834796    0.19753444]
  [ -9.834796     1.9753444 ]
  [-19.669592     3.9506888 ]
  [-19.669592     3.9506888 ]]], shape=(1, 4, 2), dtype=float32)


<IPython.core.display.Javascript object>

In [8]:
# column i corresponds to filter i
weights = c.weights[0].numpy()
weights

array([[[ 0.36053753, -0.12924767],
        [-0.32231832, -0.07781482],
        [-1.0216988 ,  0.40459692]]], dtype=float32)

<IPython.core.display.Javascript object>

In [9]:
first_filter = weights[:, :, 0].reshape(-1, 1)
second_filter = weights[:, :, 1].reshape(-1, 1)

# first filter
print(f"first_filter\n")
for i in range(4):
    print(np.dot(m[0][i], first_filter))


# second filter
print(f"\nsecond_filter\n")
for i in range(4):
    print(np.dot(m[0][i], second_filter))


first_filter

[-0.98347962]
[-9.83479619]
[-19.66959238]
[-19.66959238]

second_filter

[0.19753444]
[1.97534442]
[3.95068884]
[3.95068884]


<IPython.core.display.Javascript object>

# filters: 1
# kernel_size: 2
# strides: 1

In [10]:
c = conv_layer(m, filters=1, kernel_size=2, strides=1)

(1, 4, 3)
tf.Tensor(
[[[ -5.565763]
  [-11.70724 ]
  [-12.426883]]], shape=(1, 3, 1), dtype=float32)


<IPython.core.display.Javascript object>

In [11]:
weights = c.weights[0].numpy()
print(weights.shape)
weights

(2, 3, 1)


array([[[ 0.28502995],
        [-0.10217923],
        [-0.25481498]],

       [[-0.06151801],
        [-0.8077238 ],
        [ 0.31986195]]], dtype=float32)

<IPython.core.display.Javascript object>

In [12]:
for i in range(0, 4):
    try:
        print(np.dot(m[0][i], weights[0]) + np.dot(m[0][i + 1], weights[1]))
    except:
        pass

[-5.56576312]
[-11.70724034]
[-12.42688298]


<IPython.core.display.Javascript object>

# filters: 1
# kernel_size: 3
# strides: 1

In [13]:
c = conv_layer(m, filters=1, kernel_size=3, strides=1)

(1, 4, 3)
tf.Tensor(
[[[5.040781  ]
  [0.02628517]]], shape=(1, 2, 1), dtype=float32)


<IPython.core.display.Javascript object>

In [14]:
weights = c.weights[0].numpy()
print(weights.shape)
weights

(3, 3, 1)


array([[[ 0.23272598],
        [-0.08342904],
        [-0.20805556]],

       [[-0.05022925],
        [-0.65950376],
        [ 0.26116616]],

       [[ 0.33957738],
        [ 0.52671605],
        [-0.387033  ]]], dtype=float32)

<IPython.core.display.Javascript object>

In [15]:
print(m)

[[[ 1.  1.  1.]
  [10. 10. 10.]
  [20. 20. 20.]
  [20. 20. 20.]]]


<IPython.core.display.Javascript object>

In [16]:
for i in range(0, 4):
    try:
        print(
            np.dot(m[0][i], weights[0])
            + np.dot(m[0][i + 1], weights[1])
            + np.dot(m[0][i + 2], weights[2])
        )
    except:
        pass

[5.04078174]
[0.02628565]


<IPython.core.display.Javascript object>

# filters: 1
# kernel_size: 1
# strides: 2

In [17]:
c = conv_layer(m, filters=1, kernel_size=1, strides=2)

(1, 4, 3)
tf.Tensor(
[[[-0.10177284]
  [-2.0354564 ]]], shape=(1, 2, 1), dtype=float32)


<IPython.core.display.Javascript object>

In [18]:
weights = c.weights[0].numpy()
weights

array([[[ 0.40309322],
        [-0.14450324],
        [-0.36036283]]], dtype=float32)

<IPython.core.display.Javascript object>

<div class="alert alert-info">
    
    
It's like the example with `filters:1`, `kernel_size:1`, `strides:1`, but we skip the second and the fourth element
    
</div>

# filters: 3
# kernel_size: 2
# strides: 1

In [19]:
c = conv_layer(m, filters=5, kernel_size=2, strides=1)

(1, 4, 3)
tf.Tensor(
[[[  5.9034986  -3.710219   -8.2636795   7.812342   -3.265501 ]
  [ 13.370974   -3.467353  -17.913631   13.066171  -13.081269 ]
  [ 15.325941    1.4740026 -19.646471    9.868029  -21.269106 ]]], shape=(1, 3, 5), dtype=float32)


<IPython.core.display.Javascript object>

In [20]:
# column i  corresponds to filter i

weights = c.weights[0].numpy()
print(weights.shape)
weights

(2, 3, 5)


array([[[ 0.20154661, -0.07225162, -0.18018141, -0.04349983,
         -0.571147  ],
        [ 0.2261765 ,  0.29408258,  0.45614952, -0.3351804 ,
         -0.33901322],
        [-0.23222622,  0.27230465, -0.44925213,  0.05886596,
          0.09137678]],

       [[ 0.48951036, -0.60078186,  0.02600217,  0.1647827 ,
         -0.36824614],
        [ 0.28098398,  0.05613559, -0.48063213,  0.21641988,
          0.19629759],
        [-0.19969416,  0.12421077, -0.3544096 ,  0.4320131 ,
         -0.07272321]]], dtype=float32)

<IPython.core.display.Javascript object>

In [21]:
print(m)

[[[ 1.  1.  1.]
  [10. 10. 10.]
  [20. 20. 20.]
  [20. 20. 20.]]]


<IPython.core.display.Javascript object>

In [22]:
for i in range(0, 4):
    try:
        print(np.dot(m[0][i], weights[0][:, 0]) + np.dot(m[0][i + 1], weights[1][:, 0]))
    except:
        pass

5.903498739004135
13.370972573757172
15.325941443443298


<IPython.core.display.Javascript object>